<center> <h1>Metaclasses Demo</h1> </center>

### First-class objects
- An entity that can be constructed at run-time, passed as a parameter, returned from a function, or assigned into a variable

In [ ]:
class ExampleClass:
    pass

def return_class():
    class ClassFromFunction:
        pass
    
    return ClassFromFunction

from_func = return_class()
example_class = ExampleClass

print(from_func)
print(example_class)

#### We can create an instance of the classes

In [ ]:
instance_class_from_func = return_class()()
instance_example_class = ExampleClass()
print(instance_class_from_func)
print(instance_example_class)

#### And also assign attributes

In [ ]:
instance_class_from_func.attr1 = "My attribute 1"
instance_example_class.attr2 = "My attribute 2"
print(f"Attribute from ClassFromFunction: {instance_class_from_func.attr1}")
print(f"Attribute from ExampleClass :{instance_example_class.attr2}")

<center> <h1>Class vs Metaclass</h1> </center>

<b>Classes</b> allow us to create instances of the class. Classes define <b>rules for an object:</b> attributes, methods or operations associated with the class  

Since class is an object itself, there needs to be some higher-level class that creates it.

<b>Metaclass</b> defines <b>rules for a class</b> - Metaclass controls the creation of the class  
- The main purpose of a metaclass is to change the class automatically, when it's created.

In [ ]:
# every object in Python has a type
print(f'{type({"dict_key": 1})=}')
print(f'{type([1,2])=}')
print(f'{type((1,2))=}')
print(f'{type(ExampleClass())=}') # instance of a class
# same applies to integers, strings...

In [ ]:
# classes are object too, so what's their type?
print(f'{type(dict)=}') 
print(f'{type(list)=}')
print(f'{type(tuple)=}')
print(f'{type(ExampleClass)=}')

For instance:
- [1,2] is an instance of a **list**, and list is an instance of a **type**
- Number 100 is an instance of an **int**, and int is an instance of a **type**

In [ ]:
# At runtime we can create new list object by using list()
# or a new type object by using type()
my_list = list()
print(f'{my_list=}')

new_class = type("NewClass", (), {})
print(f"{new_class=}")

<center> <h2>'class' definition is just a syntactic sugar for calling 'type' constructor </h2> </center>

**type(name, bases, namespace)**
- name (String): name of the class to be created
- bases (Tuple): bases that the class is inheriting from
- namespace (Dict): definitions of the class body

In [ ]:
# Consider the following "normal" class example
class MyNewClass:
    attr1 = 100
    attr2 = "My string"

    def return_tuple(self):
        return (1,2)
    
    @staticmethod
    def return_list():
        return [1,2]
    
    
instance = MyNewClass()
print(f"{instance=}")
print(f"{instance.attr1=}")
print(f"{instance.attr2=}")
print(f"{instance.return_tuple()=}")
print(f"{instance.return_list()=}")

In [ ]:
# what happens behind the scenes
def behind_the_scenes():
    name = 'MyNewClass1'
    bases = () # e.g. inherit Parent class --> (Parent,) - must be tuple, hence with the comma
    
    def return_tuple(self):
        return (1,2)
    
    @staticmethod
    def return_list():
        return [1,2]
    
    namespace = {
        "attr1": 100,
        "attr2": "My string",
        "return_tuple":return_tuple,
        "return_list":return_list
    }
    return type(name, bases, namespace)

instance = behind_the_scenes()()
print(f"{instance=}")
print(f"{instance.attr1=}")
print(f"{instance.attr2=}")
print(f"{instance.return_tuple()=}")
print(f"{instance.return_list()=}")

In [ ]:
# what REALLY happens behind the scenes
def more_behind_the_scenes():
    name = 'MyNewClass2'
    bases = ()
    namespace = type.__prepare__(name, bases) # class' namespace/class dictionary is prepared
    body = (# yes, the indentation below must be like this for some reason, otherwise it throws Indentation Err
"""
attr1 = 100
attr2 = "My string"

def return_tuple(self):
    return (1,2)
    
@staticmethod
def return_list():
    return [1,2]
"""
    )
    exec(body, globals(), namespace)
    # The class's body is simply treated as code, which gets executed directly in this new dictionary namespace
    return type(name, bases, namespace)
        
instance = more_behind_the_scenes()()
print(f"{instance=}")
print(f"{instance.attr1=}")
print(f"{instance.attr2=}")
print(f"{instance.return_tuple()=}")
print(f"{instance.return_list()=}")


By specifying a custom Metaclass, we can control things such as how the class namespace is created using __prepare__ or things that happen in the __init__ or __new__ methods of the metaclass.

In [ ]:
# the default Metaclass of a class is type
class WithDefaultMeta:
    pass

print(f"{type(WithDefaultMeta)=}") # type of WithDefaultMeta is type
print(f"{WithDefaultMeta.__class__=}")
print(f"{type(WithDefaultMeta())=}") # type of instance of class is WithDefaultMeta

#### Metaclass is just the type of a type or the class of a class.

In [ ]:
class CustomMeta(type): # needs to inherit from type, otherwise TypeErr
    pass

class WithCustomMeta(metaclass=CustomMeta):
    pass

print(f"{type(WithCustomMeta)=}") # Now type of WithCustomMeta is CustomMeta
print(f"{WithCustomMeta.__class__=}")
print(f"{type(WithCustomMeta())=}")

**A Metaclass itself is quite simple:**  
 - intercepts a class creation
 - modifies the class
 - returns the modified class

In [ ]:
# Let's say we want to:
# 1) Add an "created_at" attribute for each class instance
# 2) Automatically have a "return_owner" method that returns the owner
# 3) Intercept __init__ of the AnotherClass
# for each class that uses our custom "ExampleMeta" Metaclass
from datetime import datetime

def return_owner(self):
    return "Owned by EDF"

def __custom_init__(self, description):
    self.description = description
    
# Note:
# You can have a metaclass, inheriting from other metaclasses, inheriting from type
class ExampleMeta(type):
    def __new__(mcls, name, bases, namespace):
        namespace["created_at"] = datetime.utcnow()
        namespace["return_owner"] = return_owner
        
        # calling __new__ from the parent's type class
        return super().__new__(mcls, name, bases, namespace)
    
    def __call__(cls, *args, **kwargs):
        print(f"{args=}")
        print(f"{kwargs=}")
        if kwargs:
            kwargs["description"] = "Intercepted " + kwargs["description"]
            if kwargs["max_value"] > 100:
                raise Exception("Number greater than 100 not allowed")
        return super().__call__(*args, **kwargs)
    

class AnotherClass(metaclass=ExampleMeta):
    def __init__(self, description, max_value):
        self.description = description
        self.max_value = max_value

instance = AnotherClass(description="Class description", max_value=100)
print(f'{hasattr(instance, "created_at")=}')
print(f'{hasattr(instance, "return_owner")=}')
print(f"{instance.created_at=}")
print(f"{instance.return_owner()=}")
print(f"{instance.description=}")
print(f"{instance.max_value=}")

### There is a huge problem in the way we intercept the init method and how the arguments are passed to the AnotherClass

1) Nothing is stoping the users from pasing positional arguments
 - **our init method interception in the Metaclass doesn't work!**

In [ ]:
instance = AnotherClass("Class description", 101)
print(f'{hasattr(instance, "created_at")=}')
print(f'{hasattr(instance, "return_owner")=}')
print(f"{instance.created_at=}")
print(f"{instance.return_owner()=}")
print(f"{instance.description=}") # not intercepted
print(f"{instance.max_value=}") # not intercepted, value greater than 100 which is not allowed!

2) Nothing is stopping the users from messing up the arguments completely
 - **passing someting else than a number to max_value for example**

In [ ]:
instance = AnotherClass(101, "Class description")
print(f'{hasattr(instance, "created_at")=}')
print(f'{hasattr(instance, "return_owner")=}')
print(f"{instance.created_at=}")
print(f"{instance.return_owner()=}")
print(f"{instance.description=}") # this should not be a number
print(f"{instance.max_value=}") # this certainly should not be a string

#### Metaclasses imply having complete control over the classes that are created by the Metaclass
 - restricting users from passing positional arguments and allowing keyword-only arguments for AnotherClass (we covered this in last training session)
 - fixing the call method in our Metaclass

In [ ]:
class ExampleMeta(type):
    def __new__(mcls, name, bases, namespace):
        namespace["created_at"] = datetime.utcnow()
        namespace["return_owner"] = return_owner
        
        # calling __new__ from the parent's type class
        return super().__new__(mcls, name, bases, namespace)
    
    # args are never passed since we changed the AnotherClass to only allow keyword-only arguments
    def __call__(cls, **kwargs): 
        print(f"{kwargs=}")
        # don't need if statement to verify kwargs are not empty as they will never be empty from now on
        # and our interception will always take effect
        kwargs["description"] = "Intercepted " + kwargs["description"]
        if kwargs["max_value"] > 100:
            raise Exception("Number greater than 100 not allowed")
        return super().__call__(**kwargs)
    

class AnotherClass(metaclass=ExampleMeta):
    # Everything after the * means ONLY keyword-only arguments are allowed
    def __init__(self, * , description, max_value):
        self.description = description
        self.max_value = max_value

instance = AnotherClass(description="Class description", max_value=100)

# order doesn't matter now as we can only pass keyword-only arguments
#instance = AnotherClass(max_value=100, description="Class description")

# instance = AnotherClass("Class description", 101) throws TypeError now!

print("---------")
print(f'{hasattr(instance, "created_at")=}')
print(f'{hasattr(instance, "return_owner")=}')
print(f"{instance.created_at=}")
print(f"{instance.return_owner()=}")
print(f"{instance.description=}")
print(f"{instance.max_value=}")

#### The very last part is how do we **force** users to make those init agruments keyword only?
How do we make them manually put the * in front of the arguments in the init method exactly as above?
- we create a guard in the double-underscore **new** method of the Metaclass as that method is responsible for **creating** classes

In [ ]:
import inspect

class ExampleMeta(type):
    def __new__(mcls, name, bases, namespace):
        namespace["created_at"] = datetime.utcnow()
        namespace["return_owner"] = return_owner
        
        init_params = inspect.signature(namespace["__init__"]).parameters
        for param in init_params:
            if param == "self": continue
            print(param, init_params[param].kind)
            if str(init_params[param].kind) != "KEYWORD_ONLY":
                raise Exception("Only keyword arguments allowed for __init__ method!")
        
        # calling __new__ from the parent's type class
        return super().__new__(mcls, name, bases, namespace)
    
    # args are never passed since we changed the AnotherClass to only allow keyword-only arguments
    def __call__(cls, **kwargs): 
        print(f"{kwargs=}")
        # don't need if statement to verify kwargs are not empty as they will never be empty from now on
        # and our interception will always take effect
        kwargs["description"] = "Intercepted " + kwargs["description"]
        if kwargs["max_value"] > 100:
            raise Exception("Number greater than 100 not allowed")
        return super().__call__(**kwargs)
    

class AnotherClass(metaclass=ExampleMeta):
    # Everything after the * means ONLY keyword-only arguments are allowed
    def __init__(self, * , description: str, max_value: int):
        self.description = description
        self.max_value = max_value
        

instance = AnotherClass(description="Class description", max_value=100)

# order doesn't matter now as we can only pass keyword-only arguments
#instance = AnotherClass(max_value=100, description="Class description")

# instance = AnotherClass("Class description", 101) throws TypeError now!

print("---------")
print(f'{hasattr(instance, "created_at")=}')
print(f'{hasattr(instance, "return_owner")=}')
print(f"{instance.created_at=}")
print(f"{instance.return_owner()=}")
print(f"{instance.description=}")
print(f"{instance.max_value=}")

#### If they don't make the init arguments keyword-only, the Metaclass won't allow them to create their class and exception is raised

In [ ]:
class AnotherClass(metaclass=ExampleMeta): # Throws Exception from the Metaclass!
    # Everything after the * means ONLY keyword-only arguments are allowed
    def __init__(self, description: str, max_value: int):
        self.description = description
        self.max_value = max_value
        
print("---------")
print(f'{hasattr(instance, "created_at")=}')
print(f'{hasattr(instance, "return_owner")=}')
print(f"{instance.created_at=}")
print(f"{instance.return_owner()=}")
print(f"{instance.description=}")
print(f"{instance.max_value=}")